In [1]:
import hopsworks
import joblib
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import os

2025-01-08 19:58:29,262 WARNING: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html

2025-01-08 19:58:29,372 WARNING: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages

2025-01-08 19:58:29,384 WARNING: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('ruamel')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages

2025-01-08 19:58:29,394 WARNING: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('sphinxcontrib')`.
Implementing implicit namespace packages (as specified in P

In [ ]:
import hopsworks
import os
os.environ["HOPSWORKS_API_KEY"] = "K86PR55oWoqOXhUX.VUJFWq1zEqfhZntoebWp1RLsH90VVbAYuqukvh5AcM8xHOYk7c4vg4VvD6iZEG37"
project = hopsworks.login()
feature_store = project.get_feature_store()

ModuleNotFoundError: No module named 'hopsworks'

In [6]:
spotify_features = feature_store.get_feature_group(name="spotify_features", version=2)
df = spotify_features.read()


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (36.69s) 


In [16]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors

selected_columns = ['popularity', 'artist_name', 'track_name']
training_data = df[selected_columns]

label_encoder_artist = LabelEncoder()
label_encoder_track = LabelEncoder()
training_data['artist_encoded'] = label_encoder_artist.fit_transform(training_data['artist_name'])
training_data['track_encoded'] = label_encoder_track.fit_transform(training_data['track_name'])

features = ['popularity', 'artist_encoded', 'track_encoded']
X = training_data[features]
knn_model = NearestNeighbors(n_neighbors=10, metric='cosine')
knn_model.fit(X)


def recommend_songs(user_tracks, knn_model, training_data, top_n=10):
    training_data['track_name'] = training_data['track_name'].str.strip().str.lower()
    user_tracks = [track.strip().lower() for track in user_tracks]
    filtered_tracks = training_data[training_data['track_name'].isin(user_tracks)]
    if filtered_tracks.empty:
        print("No matching tracks found for user. Using default recommendations.")
        return training_data.sort_values(by='popularity', ascending=False).head(top_n) 
    
    user_features = filtered_tracks[['popularity', 'artist_encoded', 'track_encoded']].values
    
    distances, indices = knn_model.kneighbors(user_features, n_neighbors=top_n)
    
    recommendations = training_data.iloc[indices.flatten()[:top_n]]
    return recommendations[['track_name', 'artist_name']]




2025-01-08 17:20:03,747 WARNING: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

2025-01-08 17:20:04,526 WARNING: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [17]:
#example
user_top_tracks = ['Comedy', 'Hold On','Say Something','Falling in Love at a Coffee Shop','Lucky','Left for America','あ。っという間はあるさ','Pussy Cat','Look For The Good (Single Version)'] 
recommended_songs = recommend_songs(user_top_tracks, knn_model, training_data, top_n=5)
print(recommended_songs)

2025-01-08 17:20:06,832 WARNING: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

                            track_name                    artist_name
1567                             lucky                  Tommy Genesis
244443                           masta    Vishal Dadlani, Neeti Mohan
156998      eu tive um sonho - ao vivo                     Kid Abelha
61671   love is madness (feat. halsey)  Thirty Seconds To Mars;Halsey
225175                            lost                     The Swoons


In [26]:
import os
import joblib
import hopsworks
import pandas as pd


features = ['popularity', 'artist_encoded', 'track_encoded']
X = training_data[features] 

model_dir = "./knn_model"
os.makedirs(model_dir, exist_ok=True)
model_path = os.path.join(model_dir, "knn_model.pkl")
joblib.dump(knn_model, model_path)

project = hopsworks.login()
mr = project.get_model_registry() 


metrics = {
    "Number of neighbors": 10,
}

knn_recommendation_model = mr.python.create_model(
    name="knn_recommendation_model",       
    metrics=metrics,                       
    input_example=X.iloc[0].values,        
    description=(
        "Content-based recommendation model using KNN. "
        "This model uses cosine distance with 10 neighbors."
    ),
)


knn_recommendation_model.save(model_dir)

print("KNN recommendation model has been saved to Hopsworks!")


2025-01-08 17:57:11,340 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-08 17:57:11,346 INFO: Initializing external client
2025-01-08 17:57:11,348 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-08 17:57:12,671 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164445


Uploading: 100.000%|██████████| 6705298/6705298 elapsed<00:03 remaining<00:001.37it/s]
Uploading: 100.000%|██████████| 24/24 elapsed<00:01 remaining<00:0004<00:09,  2.30s/it]
Model export complete: 100%|██████████| 6/6 [00:11<00:00,  1.97s/it]                   

Model created, explore it at https://c.app.hopsworks.ai:443/p/1164445/models/knn_recommendation_model/1
KNN recommendation model has been saved to Hopsworks!


In [28]:
mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="knn_recommendation_model",
    version=1,
)


In [4]:
import gradio as gr
import hopsworks
import joblib
import pandas as pd
from spotipy import Spotify
from spotipy.oauth2 import SpotifyOAuth
import os
from dotenv import load_dotenv
import datetime

# 加载环境变量
load_dotenv()

os.environ["HOPSWORKS_API_KEY"] = "K86PR55oWoqOXhUX.VUJFWq1zEqfhZntoebWp1RLsH90VVbAYuqukvh5AcM8xHOYk7c4vg4VvD6iZEG37"
project = hopsworks.login()
feature_store = project.get_feature_store()

# Spotify API配置
SPOTIFY_CLIENT_ID = os.getenv("SPOTIPY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = os.getenv("SPOTIPY_CLIENT_SECRET")
REDIRECT_URI = os.getenv("SPOTIPY_REDIRECT_URI")

# 全局变量存储推荐的歌曲ID
recommended_track_ids = []
spotify_client = None

def init_spotify_client():
    """初始化Spotify客户端，添加创建歌单的权限"""
    global spotify_client
    spotify_client = Spotify(auth_manager=SpotifyOAuth(
        client_id=SPOTIFY_CLIENT_ID,
        client_secret=SPOTIFY_CLIENT_SECRET,
        redirect_uri=REDIRECT_URI,
        scope="user-top-read playlist-modify-public",  # 添加创建歌单的权限
        open_browser=False
    ))
    return spotify_client

def load_recommendation_model():
    """从Hopsworks加载推荐模型"""
    project = hopsworks.login()
    mr = project.get_model_registry()
    model = mr.get_model("knn_recommendation_model", version=1)
    model_dir = model.download()
    return joblib.load(f"{model_dir}/knn_model.pkl")

def get_track_ids(sp, track_names, artist_names):
    """获取歌曲的Spotify ID"""
    track_ids = []
    for track_name, artist_name in zip(track_names, artist_names):
        results = sp.search(q=f"track:{track_name} artist:{artist_name}", type='track', limit=1)
        if results['tracks']['items']:
            track_ids.append(results['tracks']['items'][0]['id'])
    return track_ids

def get_top_tracks(sp):
    """获取用户最近播放最多的10首歌曲"""
    results = sp.current_user_top_tracks(limit=10, time_range="short_term")
    tracks = []
    for track in results['items']:
        tracks.append({
            'track_name': track['name'],
            'artist_name': track['artists'][0]['name'],
            'track_id': track['id']
        })
    return tracks

def get_recommendations(tracks, knn_model, training_data):
    """使用KNN模型获取推荐歌曲"""
    track_names = [track['track_name'] for track in tracks]
    recommendations = recommend_songs(track_names, knn_model, training_data)
    return recommendations

def spotify_auth_and_recommend():
    """Spotify授权和推荐流程"""
    global recommended_track_ids
    try:
        # 初始化Spotify客户端
        sp = init_spotify_client()
        
        # 获取用户最近播放最多的歌曲
        top_tracks = get_top_tracks(sp)
        
        # 加载推荐模型
        knn_model = load_recommendation_model()
        
        # 获取推荐
        recommendations = get_recommendations(top_tracks, knn_model, training_data)
        
        # 获取推荐歌曲的Spotify ID
        recommended_track_ids = get_track_ids(
            sp,
            recommendations['track_name'].tolist(),
            recommendations['artist_name'].tolist()
        )
        
        # 格式化输出
        output = "Your Top 10 Tracks:\n\n"
        for i, track in enumerate(top_tracks, 1):
            output += f"{i}. {track['track_name']} - {track['artist_name']}\n"
        
        output += "\nRecommended Tracks:\n\n"
        for i, (_, row) in enumerate(recommendations.iterrows(), 1):
            output += f"{i}. {row['track_name']} - {row['artist_name']}\n"
        
        return output, gr.Button.update(visible=True)
    
    except Exception as e:
        return f"An error occurred: {str(e)}", gr.Button.update(visible=False)

def create_playlist():
    """创建包含推荐歌曲的播放列表"""
    try:
        if not spotify_client or not recommended_track_ids:
            return "Please get recommendations first."
        
        # 获取用户ID
        user_id = spotify_client.current_user()['id']
        
        # 创建播放列表
        date_str = datetime.datetime.now().strftime("%Y-%m-%d")
        playlist_name = f"Recommended Songs {date_str}"
        playlist = spotify_client.user_playlist_create(
            user_id,
            playlist_name,
            public=True,
            description="Playlist created by Music Recommender based on your top tracks"
        )
        
        # 添加推荐的歌曲
        spotify_client.playlist_add_items(playlist['id'], recommended_track_ids)
        
        return f"Successfully created playlist '{playlist_name}' with recommended songs! Check your Spotify account."
    
    except Exception as e:
        return f"Error creating playlist: {str(e)}"

# 创建Gradio界面
with gr.Blocks() as iface:
    gr.Markdown("# Spotify Music Recommender")
    gr.Markdown("Get personalized music recommendations based on your listening history.")
    
    with gr.Row():
        get_recommendations_btn = gr.Button("Get Recommendations")
        create_playlist_btn = gr.Button("Create Playlist", visible=False)
    
    output_text = gr.Textbox(label="Results")
    
    get_recommendations_btn.click(
        fn=spotify_auth_and_recommend,
        outputs=[output_text, create_playlist_btn]
    )
    
    create_playlist_btn.click(
        fn=create_playlist,
        outputs=output_text
    )

# 启动应用
if __name__ == "__main__":
    iface.launch(share=True)

2025-01-08 18:57:35,944 INFO: Initializing external client
2025-01-08 18:57:35,945 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-08 18:57:37,707 INFO: Python Engine initialized.


Exception: Couldn't find client. Try reconnecting to Hopsworks.

In [3]:
!pip install gradio

  Using cached gradio-4.44.1-py3-none-any.whl.metadata (15 kB)
  Using cached aiofiles-23.2.1-py3-none-any.whl.metadata (9.7 kB)
  Using cached ffmpy-0.5.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached gradio_client-1.3.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached python_multipart-0.0.20-py3-none-any.whl.metadata (1.8 kB)
  Using cached ruff-0.8.6-py3-none-macosx_10_12_x86_64.whl.metadata (25 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl.metadata (9.7 kB)
  Using cached tomlkit-0.12.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached typer-0.15.1-py3-none-any.whl.metadata (15 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metad